<a href="https://colab.research.google.com/github/markuskunej/air-pollution-thesis/blob/master/VARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#DATA_PATH = "/content/drive/MyDrive/Air_Pollution_Data/Y&E_60m_last40days"
#!unzip "/content/drive/MyDrive/Air_Pollution_Data/Y&E_60m_last40days.zip"

In [ ]:
import pandas as pd
import os
import glob

DATA_PATH = "/content/drive/MyDrive/Air_Pollution_Data/Y&E_60m_last40days"
print(os.path.join(DATA_PATH , "/*.csv"))
all_files = glob.glob(DATA_PATH + '/*.csv')
keys = []
dfs = []
big_df = []
for file_name in all_files:
  df = pd.read_csv(file_name, parse_dates=["Time"])
  variable_name = os.path.basename(file_name).split(".")[0]
  df[variable_name] = df.mean(axis=1)
  #keys.append(variable_name)
  df.set_index('Time', inplace=True)
  df.info()
  df = df.tz_localize(tz='US/Eastern', ambiguous='infer')
  df.info()
  print(df[df.index.duplicated(keep=False)])

  # reduce dataframe columns to only the average
  df = df[[variable_name]]

  print(df.index)
  dfs.append(df)

# concat dataframes into one
big_df = pd.concat(dfs, axis=1)
print(big_df.shape)
print(big_df.head())

In [ ]:
#see how many nan values exist now
print(big_df.isnull().sum())

# delete rows from beginning and end that contain NaN values (since date range for each variable is not the same)

# drop nan rows from beginning
while(big_df.iloc[0].isnull().values.any() == True):
  big_df.drop(index=big_df.index[0], axis=0, inplace=True)

#drop nan rows from end
while(big_df.iloc[-1].isnull().values.any() == True):
  big_df.drop(index=big_df.index[-1], axis=0, inplace=True)

#see how many nan values exist after slicing the beginning and end of dataframe
print(big_df.isnull().sum())


Plot Current Data

In [ ]:
import matplotlib.pyplot as plt

def plot_df(df):
  plot_cols = df.columns
  fig,ax = plt.subplots(len(plot_cols), figsize=(20,40), sharex=True)
  df.plot(subplots=True, legend=False, ax=ax)
  for a in range(len(ax)): 
      ax[a].set_ylabel(plot_cols[a])
  ax[-1].set_xlabel('')
  plt.tight_layout()
  plt.show()

plot_df(big_df)

Remove unhelpful variables

In [ ]:
# based on the graphs, remove latitude and longitude.
# Also remove AQI since this number is determined based on pollutant levels (which we already have)
big_df.drop(['AQI', 'Latitude', 'Longitude', 'Elevation'], axis=1, inplace=True)

Augmented Dickey-Fuller Test

In [ ]:
# test for stationarity, difference if seasonality exists
# https://michael-fuchs-python.netlify.app/2020/10/29/time-series-analysis-regression-extension-techniques-for-forecasting-multivariate-variables/#stationarity
from statsmodels.tsa.stattools import adfuller

def Augmented_Dickey_Fuller_Test_func(timeseries , column_name):
    '''
    Calculates statistical values whether the available data are stationary or not 
    
    Args:
        series (float64): Values of the column for which stationarity is to be checked, numpy array of floats 
        column_name (str): Name of the column for which stationarity is to be checked
    
    Returns:
        p-value that indicates whether the data are stationary or not
    ''' 
    print (f'Results of Dickey-Fuller Test for column: {column_name}')
    adfTest = adfuller(timeseries, autolag='AIC')   # why AIC vs BIC, t-stat, etc.?
    dfResults = pd.Series(adfTest[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
    for key, value in adfTest[4].items():
       dfResults['Critical Value (%s)'%key] = value
    print (dfResults)
    if adfTest[1] <= 0.05:
        print()
        print("Conclusion:")
        print("Reject the null hypothesis")
        print('\033[92m' + "Data is stationary" + '\033[0m')
    else:
        print()
        print("Conclusion:")
        print("Fail to reject the null hypothesis")
        print('\033[91m' + "Data is non-stationary" + '\033[0m')


# Check each column for seasonality
for name, column in big_df.iteritems():
    Augmented_Dickey_Fuller_Test_func(big_df[name],name)
    print('\n')



Since not all variables are stationary, we must perform co-integration test (apparently I could have skipped the dickey-fuller test?)

In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen   #can't use on more than 12 variables

johansenResults = coint_johansen(big_df.iloc[:,:12],-1,1)

print("Trace Stat:")
print(johansenResults.trace_stat)
print("\nTrace Stat Crit Vals:")
print(johansenResults.trace_stat_crit_vals)
print("\nMax Eig stat:")
print(johansenResults.max_eig_stat)
print("\nMax Eig Stat Crit Vals:")
print(johansenResults.max_eig_stat_crit_vals)


Difference the non-stationary variables

In [ ]:
non_stationary_columns = ["PM10", "CO2", "CO", "Temperature"]

# visualize cols before
plot_df(big_df[non_stationary_columns])

In [ ]:
# difference non-stationary variables
transformed_big_df = big_df.copy()
transformed_big_df[non_stationary_columns] = transformed_big_df[non_stationary_columns].apply(lambda x: x.diff())

# drop nan rows from beginning, differencing produces a NaN for first value
while(transformed_big_df.iloc[0].isnull().values.any() == True):
  transformed_big_df.drop(index=transformed_big_df.index[0], axis=0, inplace=True)

In [ ]:
# visualize after
plot_df(transformed_big_df[non_stationary_columns])

In [ ]:
# run the adf test to check for stationary data

# Check each column for seasonality
for name, column in transformed_big_df.iteritems():
    Augmented_Dickey_Fuller_Test_func(transformed_big_df[name],name)
    print('\n')

Granger Causality Test

In [ ]:
#https://blogs.sap.com/2021/05/06/a-multivariate-time-series-modeling-and-forecasting-guide-with-python-machine-learning-client-for-sap-hana/
from statsmodels.tsa.stattools import grangercausalitytests
import numpy as np

maxlag_ = 20
variables = transformed_big_df.columns
matrix = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
for col in matrix.columns:
    for row in matrix.index:
        test_result = grangercausalitytests(transformed_big_df[[row, col]], maxlag=maxlag_, verbose=False)            
        p_values = [round(test_result[i+1][0]['ssr_chi2test'][1],4) for i in range(maxlag_)]            
        min_p_value = np.min(p_values)
        matrix.loc[row, col] = min_p_value
matrix.columns = [var + '_x' for var in variables]
matrix.index = [var + '_y' for var in variables]
print(matrix)

In [ ]:
#creating the train and validation set
train_size = 0.7

transformed_train_df = transformed_big_df[:int(train_size*(len(transformed_big_df)))]
train_df = big_df[:int(train_size*(len(big_df)))]
print(len(train_df))
valid_df = big_df[int(train_size*(len(big_df))):]
print(len(valid_df))

Get best AR terms

In [ ]:

from statsmodels.tsa.api import VAR
from statsmodels.tsa.statespace.varmax import VARMAX

## MODEL RUN NUMBER ##
RUN_ID = 1

# used to select best AIC lag order
model = VAR(transformed_big_df)
sorted_order=model.select_order(maxlags=20)
print(sorted_order.summary())

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'


 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0        37.55       37.63   2.023e+16       37.58
1        5.546       6.950       256.1       6.081
2        3.225      5.952*       25.17      4.265*
3       3.015*       7.063      20.42*       4.558
4        3.098       8.468       22.22       5.145
5        3.091       9.784       22.16       5.643
6        3.108       11.12       22.63       6.163
7        3.244       12.58       26.13       6.804
8        3.342       14.00       29.08       7.405
9        3.508       15.49       34.74       8.075
10       3.701       17.00       42.79       8.773
11       3.769       18.39       46.61       9.344
12       3.906       19.85       54.68       9.986
13       4.026       21.29       63.28       10.61
14       4.169       22.76       75.27       11.26
15       4.315       24.23       90.36       11.91
16       4.420       25.65     

In [ ]:
# use the non differenced df since VARMAX can do its own automatic differencing
# second order is 0 since we're not using moving average here (MAX part of VARMAX)
var_model = VARMAX(train_df, order=(3,0), enforce_stationarity=True)
fitted_model = var_model.fit(disp=False)

#save model for future reference
fitted_model.save('/content/drive/MyDrive/Air_Pollution_Models/{}_run.png'.format(RUN_ID))
print(fitted_model.summary())

In [ ]:
min_interval = 60 #change depending on data interval
n_day_forecast = int(1440 / min_interval)
print(n_day_forecast)
n_week_forecast = int(n_day_forecast*7)
n_2week_forecast = int(n_week_forecast*2)
n_month_forecast = int(n_2week_forecast*2)
predict_day = fitted_model.get_prediction(start=len(train_df), end=len(train_df) + n_day_forecast - 1)
predict_week = fitted_model.get_prediction(start=len(train_df), end=len(train_df) + n_week_forecast - 1)
predict_2week = fitted_model.get_prediction(start=len(train_df), end=len(train_df) + n_2week_forecast - 1)
predict_month = fitted_model.get_prediction(start=len(train_df), end=len(train_df) + n_month_forecast - 1)

predictions_day=predict_day.predicted_mean.add_suffix('_Prediction')
predictions_week=predict_week.predicted_mean.add_suffix('_Prediction')
predictions_2week=predict_2week.predicted_mean.add_suffix('_Prediction')
predictions_month=predict_month.predicted_mean.add_suffix('_Prediction')

24


In [ ]:
valid_vs_pred_day_df = pd.concat([valid_df.iloc[:n_day_forecast], predictions_day], axis=1)
valid_vs_pred_week_df = pd.concat([valid_df.iloc[:n_week_forecast], predictions_week], axis=1)
valid_vs_pred_2week_df = pd.concat([valid_df.iloc[:n_2week_forecast], predictions_2week], axis=1)
valid_vs_pred_month_df = pd.concat([valid_df.iloc[:n_month_forecast], predictions_month], axis=1)

In [ ]:
from statsmodels.tsa.stattools import acf
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'corr':corr, 'minmax':minmax})


In [ ]:
pred_columns = ["PM1", "PM2", "PM10", "CO", "CO2", "O3", "NO", "NO2"]
def adjust(val, length= 6): 
  return str(val).ljust(length)

metrics_df = pd.DataFrame()

for pred_range_and_df in [("Day", valid_vs_pred_day_df), ("Week", valid_vs_pred_week_df), ("2_weeks", valid_vs_pred_2week_df), ("Month", valid_vs_pred_month_df)]:
  for col in pred_columns:
    print('\nForecast accuracy of ' + col)
    pred_range, pred_df = pred_range_and_df
    accuracy_prod = forecast_accuracy(pred_df[col + '_Prediction'].values, pred_df[col])
    for k, v in accuracy_prod.items():
      print(adjust(k), ': ', round(v,4))
      metrics_df.at[pred_range, col, k] = v

print(metrics_df)

In [ ]:
import matplotlib.pyplot as plt 

pred_columns = ["PM1", "PM2", "PM10", "CO", "CO2", "O3", "NO", "NO2"]
fig, axes = plt.subplots(len(pred_columns), 2, figsize=(24, 50))
i = 0
for column in pred_columns:
  valid_vs_pred_day_df.plot(y=[column, column + "_Prediction"], ax=axes[i,0])
  valid_vs_pred_week_df.plot(y=[column, column + "_Prediction"],ax=axes[i,1])
  i = i + 1
  
plt.savefig('/content/drive/MyDrive/Air_Pollution_Prediction_Figures/{}_run.png'.format(RUN_ID))

In [ ]:
AR_Term_value_VAR = best_values_VAR['AR_Term'].iloc[0]

print("AR_Term_value_VAR: ", AR_Term_value_VAR)

In [ ]:
from hana_ml.algorithms.pal.tsa.vector_arima import VectorARIMA

vectorArima1 = VectorARIMA(order=(-1, 2, -1), model_type = 'VARMA', search_method='grid_search', output_fitted=True, max_p=5, max_q=5)
vectorArima1.fit(data=train)

In [ ]:
#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

In [ ]:
# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

In [ ]:
#converting predictions to dataframe
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,13):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]